In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
base_dir = os.getcwd()

## Load data 

In [2]:
data_filename = "m29_7arcmin_panstarrs_data.csv"
data = pd.read_csv(os.path.join(base_dir, data_filename), sep=';')
useful_columns = ["_r", "RAJ2000", "DEJ2000", "gmag", "e_gmag", "rmag", "e_rmag", "imag", "e_imag", "zmag", "e_zmag"]
data = data[useful_columns]

def to_float(x): 
    try: 
        return float(x)
    except Exception:
        return np.nan 
data = data.applymap(to_float).astype(float)
print(data.describe())

                _r      RAJ2000      DEJ2000         gmag       e_gmag  \
count  6103.000000  6103.000000  6103.000000  2775.000000  1760.000000   
mean      4.713027   305.933606    38.482211    21.518668     0.059655   
std       1.622350     0.074891     0.058212     3.321451     0.068847   
min       0.034900   305.794028    38.370887     8.725000     0.000000   
25%       3.582200   305.872763    38.433972    19.862950     0.009475   
50%       4.991800   305.929423    38.481161    21.577800     0.030750   
75%       6.088200   305.992937    38.528819    23.208000     0.081100   
max       6.999900   306.091144    38.602637    30.090000     0.432200   

              rmag       e_rmag         imag       e_imag         zmag  \
count  3702.000000  3187.000000  5594.000000  5401.000000  5895.000000   
mean     20.065846     0.061443    19.683335     0.050423    19.009028   
std       2.377252     0.069253     1.994553     0.058106     1.806394   
min       7.983500     0.000000     8

In [3]:
data.dropna(subset=["gmag", "rmag", "imag", "zmag"], inplace=True)

In [6]:
def ugriz_to_ubvri(ugriz):
    u, g, r, i, z = ugriz
    u = np.array(u)
    g = np.array(g)
    r = np.array(r)
    i = np.array(i)
    z = np.array(z)

    V = g - 0.59*(g - r) - 0.01
    B = g + 0.39*(g - r) + 0.21
    R = V - (1.09*(r - i) + 0.22)
    U = 0.77*(u - g) - 0.88 + B

    I = R - (r - i + 0.21)
    return (U, B, V, R, I)

gmags = data["gmag"]
umags = pd.Series([69 for _ in range(len(gmags))])
rmags = data["rmag"]
imags = data["imag"]
zmags = data["zmag"]
_, data["B"], data["V"], data["R"], data["I"] = ugriz_to_ubvri((umags, gmags, rmags, imags, zmags))
print(data.describe())


                _r      RAJ2000      DEJ2000         gmag       e_gmag  \
count  2641.000000  2641.000000  2641.000000  2641.000000  1629.000000   
mean      4.725604   305.935539    38.483476    21.623468     0.055435   
std       1.592680     0.075223     0.058280     3.276687     0.065481   
min       0.034900   305.794028    38.371720     8.725000     0.000000   
25%       3.592300   305.873864    38.435752    19.892500     0.009100   
50%       4.972200   305.929754    38.482309    21.618000     0.028400   
75%       6.075000   305.994990    38.529979    23.485000     0.070900   
max       6.999800   306.091144    38.602637    30.090000     0.432200   

              rmag       e_rmag         imag       e_imag         zmag  \
count  2641.000000  2131.000000  2641.000000  2451.000000  2641.000000   
mean     19.719195     0.036523    18.562242     0.022226    17.803363   
std       2.478741     0.049236     2.039345     0.034142     1.803760   
min       8.338800     0.000000     8

## Apply distance correction, reddening

In [7]:
m29_dist = 1148 # pc
m39_dist = 326
obs_minus_absolue = 5*np.log10(m29_dist) - 5
data["B"] -= obs_minus_absolue
data["V"] -= obs_minus_absolue
data["R"] -= obs_minus_absolue
data["I"] -= obs_minus_absolue
a_redd = 0.744

A_v = 3.1*a_redd
A_b = 1.32*A_v
A_r = 0.79*A_v
A_i = 0.55*A_v

data["B"] -= A_b
data["V"] -= A_v 
data["R"] -= A_r
data["I"] -= A_i
print(len(data["B"]))

2641


## Load isochrones

In [8]:
times = [6.0 + i*0.1 for i in range(31)]

filenames = [f"isochrone_{10*t:.0f}.dat" for t in times]
isochrone_cols = ["M_ini", "MV", "U-B", "B-V", "V-R", "V-I"]
isochrones = {
    times[i]: pd.read_csv(os.path.join("..", "isochrones", filenames[i]), delim_whitespace=True)[isochrone_cols] for i in range(len(filenames))
}

low_mass_filename = [f"lm_isochrone_{10*t:.0f}.dat" for t in times]
lm_iso_cols = ["M_ini","M_U", "M_B", "M_V", "M_R", "M_I"]
low_mass_isochrones = {
    times[i]: pd.read_csv(os.path.join("..", "isochrones", low_mass_filename[i]), delim_whitespace=True)[lm_iso_cols] for i in range(len(filenames))
}
print(low_mass_isochrones[6].columns)


Index(['M_ini', 'M_U', 'M_B', 'M_V', 'M_R', 'M_I'], dtype='object')


## Shifted isochrone

In [9]:
# plt.tight_layout()
# plt.scatter(data["B"] - data["V"], data["V"] + obs_minus_absolue, s=1)
# plt.show()

In [10]:
%matplotlib qt
from matplotlib import colors

times = [6.0 + 0.1*i for i in range(31)]
final_isochrones = {}
for index, log_t in enumerate(times):
    vmags = isochrones[log_t]["MV"] 
    bmags = isochrones[log_t]["B-V"] + vmags
    rmags = vmags - isochrones[log_t]["V-R"]
    imags = vmags - isochrones[log_t]["V-I"]
    umags = isochrones[log_t]["U-B"] + bmags 
    isochrones[log_t]["isoU"] = umags
    isochrones[log_t]["isoB"] = bmags
    isochrones[log_t]["isoV"] = vmags
    isochrones[log_t]["isoR"] = rmags
    isochrones[log_t]["isoI"] = imags
    # add data from random table

    low_mass = pd.DataFrame(columns=["M_ini", "isoU", "isoB", "isoV", "isoR", "isoI"], data=[
        [0.47, 12.55, 11.35, 9.85, 8.87, 7.76], 
        [0.44, 12.88, 11.71, 10.21, 9.21, 8.04], 
        [0.40, 13.305, 12.13, 10.61, 9.57, 8.3], 
        [0.37, 13.86, 12.68, 11.15, 10.07, 8.73], 
        [0.27, 14.9, 13.7, 12.1, 10.32, 9.42], 
    ])

    final_isochrones[log_t] = isochrones[log_t][["M_ini", "isoU", "isoB", "isoV", "isoR", "isoI"]]
    final_isochrones[log_t] = pd.concat((final_isochrones[log_t], low_mass))

    # lmu = low_mass_isochrones[log_t]["M_U"]
    # lmb = low_mass_isochrones[log_t]["M_B"]
    # lmv = low_mass_isochrones[log_t]["M_V"]
    # lmr = low_mass_isochrones[log_t]["M_R"]
    # lmi = low_mass_isochrones[log_t]["M_I"]

    # low_mass_isochrones[log_t]["isoU"] = lmu 
    # low_mass_isochrones[log_t]["isoB"] = lmb 
    # low_mass_isochrones[log_t]["isoV"] = lmv 
    # low_mass_isochrones[log_t]["isoR"] = lmr 
    # low_mass_isochrones[log_t]["isoI"] = lmi 
    
    # low_mass_isochrones[log_t] = low_mass_isochrones[log_t][low_mass_isochrones[log_t]["M_ini"] < 0.5]  
    # final_isochrones[log_t] = pd.concat((low_mass_isochrones[log_t], isochrones[log_t]))
    final_isochrones[log_t].sort_values(by="M_ini", inplace=True)

def plot_3_cmds(data):
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3)
    B = data["B"]
    V = data["V"]
    R = data["R"]
    I = data["I"]

# ax1 B-V vs V
    ax1.scatter(B-V, V, s=1)
    ax1.set_xlabel("B-V")
    ax1.set_ylabel("V")

# ax2 V-R vs R
    ax2.scatter(V-R, R, s=1)
    ax2.set_xlabel("V-R")
    ax2.set_ylabel("R")

# ax3 R-I vs I
    ax3.scatter(R-I, I, s=1)
    ax3.set_xlabel("R-I")
    ax3.set_ylabel("I")

    plt.tight_layout()
# isochrone
    cdict = {'red':   ((0.0,  0.22, 0.0),
                       (0.5,  1.0, 1.0),
                       (1.0,  0.89, 1.0)),

             'green': ((0.0,  0.49, 0.0),
                       (0.5,  1.0, 1.0),
                       (1.0,  0.12, 1.0)),

             'blue':  ((0.0,  0.72, 0.0),
                       (0.5,  0.0, 0.0),
                       (1.0,  0.11, 1.0))}

    cmap = colors.LinearSegmentedColormap('custom', cdict)

    times = [6.0 + 0.1*i for i in range(31)]
    for index, log_t in enumerate(times):
        # vmags = isochrones[log_t]["MV"] 
        # bmags = isochrones[log_t]["B-V"] + vmags
        # rmags = vmags - isochrones[log_t]["V-R"]
        # imags = vmags - isochrones[log_t]["V-I"]
        # umags = isochrones[log_t]["U-B"] + bmags 
        # isochrones[log_t]["isoU"] = umags
        # isochrones[log_t]["isoB"] = bmags
        # isochrones[log_t]["isoV"] = vmags
        # isochrones[log_t]["isoR"] = rmags
        # isochrones[log_t]["isoI"] = imags
        ax1.plot(bmags - vmags, bmags, c=cmap(index/len(times)))
        ax2.plot(vmags - rmags, vmags, c=cmap(index/len(times)))
        ax3.plot(rmags - imags, rmags, c=cmap(index/len(times)))
    ax1.set_xlim((-0.5, 2.5))
    ax1.set_ylim((-2, 7.5))

    ax2.set_xlim((-0.5, 2))
    ax2.set_ylim((-2, 7))

    ax3.set_ylim((-2, 6.5))
    ax3.set_xlim((-1, 1.5))
    ax1.invert_yaxis()
    ax2.invert_yaxis()
    ax3.invert_yaxis()
    
print(final_isochrones[6].describe())

           M_ini       isoU       isoB       isoV       isoR       isoI
count  56.000000  56.000000  56.000000  56.000000  56.000000  56.000000
mean    1.447196   5.710075   5.324430   4.732530   4.356632   4.022557
std     0.894317   4.414720   3.844301   3.267002   2.888168   2.520086
min     0.270000  -0.130100   0.312800   0.452500   0.501300   0.618100
25%     0.703250   1.737275   1.784425   1.779900   1.759325   1.761000
50%     1.200500   4.877200   4.774850   4.244900   3.940550   3.663850
75%     2.050000   9.809425   8.663950   7.475625   6.798900   6.201275
max     3.500000  14.900000  13.700000  12.100000  10.320000   9.420000


## Metric in UBVRI space 
attempt 1: metric is of data scaled so that its std is 1 (approx the same shape in all dimensions)

In [11]:
def metric1(color1, color2, stds):
    """
    color is the tuple (U,B,V,R,I), or (B, V, R, I)
    stds is a list of the same shape as colors, and contains the stds of the corresponding colors
    """
    return sum([((color1[i] - color2[i])/stds[i])**2 for i in range(len(color1))])**0.5

In [13]:
from matplotlib import colors
%matplotlib qt
fig = plt.figure()
ax = plt.axes(projection="3d")
# catalogue data
# outliers identified by eye
data = data[data["B"] > 0]
data = data[data["V"] > 0]
data = data[data["R"] > 0]
B = data["B"]
V = data["V"]
R = data["R"]
I = data["I"]
ax.scatter3D(B, V, R, s=0.5, color='red')

# isochrone
cdict = {'red':   ((0.0,  0.22, 0.0),
                   (0.5,  1.0, 1.0),
                   (1.0,  0.89, 1.0)),

         'green': ((0.0,  0.49, 0.0),
                   (0.5,  1.0, 1.0),
                   (1.0,  0.12, 1.0)),

         'blue':  ((0.0,  0.72, 0.0),
                   (0.5,  0.0, 0.0),
                   (1.0,  0.11, 1.0))}

cmap = colors.LinearSegmentedColormap('custom', cdict)

times = [7.1]
for index, log_t in enumerate(times):
    bmags = final_isochrones[log_t]["isoB"]
    vmags = final_isochrones[log_t]["isoV"]
    rmags = final_isochrones[log_t]["isoR"]
    imags = final_isochrones[log_t]["isoI"]

    ax.plot(bmags, vmags, rmags, c=cmap(index/len(times)))
ax.set_xlabel("B mag")
ax.set_ylabel("V mag")
ax.set_zlabel("R mag")
plt.title("Изохрона за $t=10^7.1$ yr", fontsize=20)
plt.show()

2606
2606


## Plot n_stars below a certain distance 

In [111]:
stds = [np.std(data[filter]) for filter in ["B", "V", "R", "I"]]
right_isochrone = final_isochrones[8.5]

distance_data = []
mass_data = []
isochrone_compressed = right_isochrone[["isoB", "isoV", "isoR", "isoI"]]
print(right_isochrone.describe())
for i in range(len(data["B"])):
    if i%100 == 0:
        print(i)
    star_coords = list(data[["B", "V", "R", "I"]].iloc[i]) 

    array_of_distances = [
        metric1(
            star_coords,
            list(isochrone_compressed.iloc[j]), 
            stds
        ) for j in range(len(right_isochrone["isoB"]))
    ]
    min_index = np.argmin(array_of_distances)
    min_distance = array_of_distances[min_index] 

    mass_data.append(right_isochrone["M_ini"].iloc[min_index])
    distance_data.append(min_distance)
print("done")
data["distance"] = distance_data
data["mass"] = mass_data

            M_ini        isoU        isoB        isoV        isoR        isoI
count  163.000000  163.000000  163.000000  163.000000  163.000000  163.000000
mean     2.506546    1.517642    1.436977    1.191951    1.028648    0.888241
std      0.927765    3.981640    3.589535    3.173833    2.914880    2.673538
min      0.270000   -0.920100   -0.813800   -0.872400   -0.949500   -1.120900
25%      1.991000   -0.680450   -0.528300   -0.565400   -0.643700   -0.679000
50%      3.057000   -0.570800   -0.444700   -0.447800   -0.469100   -0.466900
75%      3.077000    1.789450    1.751450    1.708350    1.670450    1.638900
max      3.097000   14.900000   13.700000   12.100000   10.320000    9.420000
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100


In [112]:
print(data.describe())

                 _r       RAJ2000       DEJ2000          gmag        e_gmag  \
count  14166.000000  14166.000000  14166.000000  14166.000000  14055.000000   
mean       9.516636    322.957221     48.441161     19.419095      0.021660   
std        3.250926      0.176036      0.119876      1.648941      0.025953   
min        0.224400    322.599338     48.201035      7.689000      0.000000   
25%        7.239250    322.816100     48.342521     18.603400      0.006800   
50%       10.143800    322.956058     48.445144     19.818800      0.014400   
75%       12.252075    323.102131     48.541192     20.660075      0.027300   
max       14.000000    323.301227     48.666350     21.904800      0.452700   

               rmag        e_rmag          imag        e_imag          zmag  \
count  14166.000000  14013.000000  14166.000000  13969.000000  14166.000000   
mean      18.460630      0.010899     17.934821      0.008557     17.640446   
std        1.584687      0.015275      1.530051    

In [159]:
%matplotlib qt
data = data[data["distance"] < 1]
n_bins = 10 
bins = np.logspace(-0.56, 0.7, n_bins)
hist_data, bins = np.histogram(data["mass"], bins=bins)
log_bins = np.array([np.log(b) for b in bins[:-1]])
log_hd = np.array([np.log(hd) if hd != 0 else 0 for hd in hist_data])
plt.scatter(log_bins, log_hd)

a, b = np.polyfit(log_bins, log_hd, 1)
print(f"a: {a}, b: {b}")
# plt.yscale('log')
# plt.xscale('log')
# plt.hist(data["distance"], bins=30, range=[0, 5])
# plt.hist(data["mass"], bins=60, range=[0.2, 4])
plt.plot(log_bins, a*log_bins+b)
plt.show()

# print(data.describe())

a: -3.374521566192989, b: 4.023433925850827
